In [7]:
# Import the relevant modules

import numpy as np
import pandas as pd
import matplotlib as plt
%matplotlib inline
import talib as ta

In [2]:
# Load the data. If you do not have any bitmex data yet, you can use the downloader 
# in the Bitmex Historical repository https://github.com/fractalswift/bitmex-historical

# Please ensure data matches the format below

data = pd.read_pickle('pickled_5m_bitmex/ETHUSD_5m')
data = data[['timestamp', 'open', 'high', 'low', 'close']]
data.tail()

,timestamp,open,high,low,close
83245,2019-06-15 01:05:00,264.60,264.85,264.00,264.45
83246,2019-06-15 01:10:00,264.45,264.55,264.30,264.35
83247,2019-06-15 01:15:00,264.35,264.90,263.95,264.85
83248,2019-06-15 01:20:00,264.85,264.95,264.55,264.75
83249,2019-06-15 01:25:00,264.75,265.70,264.80,265.45


In [3]:
# Generate the signal 1 for buy, -1 for sell or 0 for do nothing.

# In this case we use price crossing simple moving average as a signal

# Create the SMA using pandas rolling function. In this case we have set it to a period of 20

data['SMA'] = data['close'].rolling(20).mean()

# These filters ensure the signal is only generated when price crossing up/down or down/up over the SMA

price_above_SMA = data.close > data.SMA
price_below_SMA = data.close < data.SMA

prev_price_above_SMA = data.close.shift(1) > data.SMA.shift(1)
prev_price_below_SMA = data.close.shift(1) < data.SMA.shift(1)

is_green = data.close > data.open
is_red = data.close < data.open

# Add in the signal when criteria is met

data['signal'] = np.where(prev_price_below_SMA & price_above_SMA, 1, 0)
data['signal'] = np.where(prev_price_above_SMA & price_below_SMA, -1, data.signal)

# Clean the data - we only need the price info and signal info 

data = data[['timestamp', 'open', 'high', 'low', 'close', 'signal']]

data['signal'].value_counts()

# IMPORTANT - do I have to shift to avoid future bias? Not sure I do if I'm using market..

 0    72681
 1     5285
-1     5284
Name: signal, dtype: int64

In [4]:
# This is the main backtest logic. You do not need to change this, but notice the variables required for the function

# _df: The dataframe with price and signals you have created above
# _direction: Can use this for only long/short. Leave as 1 unless you want to modify the script
# _stop: Your stop loss. For example, 0.99 
# _tp: Your take profit. For example, 1.01
# _time_in_trade: How many periods you want to leave the trade active. Please note, the longer you set this, the
# longer the backtet will take

def backtest_run(_df, _direction, _stop, _tp, _time_in_trade):
    
    trades_list = []
    
    for index, row in _df.iterrows():
        
        if row.signal == 1:
            
            trade_window = _df.iloc[index:index + _time_in_trade].values.tolist()
            
            stop = _stop * trade_window[0][4]
            tp = _tp * trade_window[0][4]
            
           
            for win_row in trade_window:
                
                high = win_row[2]
                low = win_row[3]
                
                trade_row = win_row
                
                if low < stop:
                    trade_result = -1
                    trade_row.append(trade_result)
                    continue
                if high > tp:
                    trade_result = 1
                    trade_row.append(trade_result)
                    continue
                else:
                    trade_result = 0
                    trade_row.append(trade_result)
                    
                
            
            
            trades_list.append(trade_row)
        
        
        if row.signal == -1:
            
            trade_window = _df.iloc[index:index + _time_in_trade].values.tolist()

            stop = (( 1 -_stop) + 1) * trade_window[0][4]
            tp = (2 - _tp) * trade_window[0][4]


            for win_row in trade_window:

                high = win_row[2]
                low = win_row[3]

                trade_row = win_row

                if high > stop:
                    trade_result = -1
                    trade_row.append(trade_result)
                    continue
                if low < tp:
                    trade_result = 1
                    trade_row.append(trade_result)
                    continue
                else:
                    trade_result = 0
                    trade_row.append(trade_result)




            trades_list.append(trade_row)

    return trades_list
    

In [5]:

# Run the backtest
trades = backtest_run(data, 1, 0.99, 1.013, 20)
trades

[[Timestamp('2018-08-30 05:00:00'), 286.6, 287.25, 286.6, 287.1, 1, 0],
 [Timestamp('2018-08-30 05:05:00'), 287.1, 287.15, 286.85, 287.0, 0, 0],
 [Timestamp('2018-08-30 05:10:00'), 287.0, 286.95, 286.1, 286.65, -1, 0],
 [Timestamp('2018-08-30 05:15:00'), 286.65, 286.7, 286.6, 286.6, 0, 0],
 [Timestamp('2018-08-30 05:30:00'), 286.05, 286.0, 285.3, 285.35, 0, 0],
 [Timestamp('2018-08-30 05:55:00'), 285.0, 284.95, 284.35, 284.55, 0, 0],
 [Timestamp('2018-08-30 06:35:00'), 284.5, 284.5, 283.45, 283.65, 0, -1],
 [Timestamp('2018-08-30 06:45:00'), 282.85, 283.5, 282.65, 283.5, 0, 1],
 [Timestamp('2018-08-30 08:45:00'), 282.55, 282.55, 281.45, 281.95, 0, 0],
 [Timestamp('2018-08-30 08:50:00'), 281.95, 282.05, 280.9, 280.95, 0, 0],
 [Timestamp('2018-08-30 08:55:00'), 280.95, 281.75, 280.65, 281.05, 0, -1],
 [Timestamp('2018-08-30 09:25:00'), 280.65, 280.7, 279.65, 280.25, 0, 1],
 [Timestamp('2018-08-30 12:25:00'), 277.65, 277.9, 277.3, 277.75, 0, 0],
 [Timestamp('2018-08-30 13:00:00'), 276.75,

In [ ]:
# Check the amount of signals...

data['signal'].value_counts()

In [6]:
# This function gives you more detail on the results. Make sure the values match the values you used in the test.

def analyse_results(_trades_list, _fees, _position_size, _stop, _tp):
    
    trades_df = pd.DataFrame(_trades_list, columns=['ts', 'o', 'h', 'l', 'c', 'signal', 'result'])
    
    win = np.where(trades_df.result == 1, 1, 0)
    lose = np.where(trades_df.result == -1, 1, 0)
    
    attempt_trade = np.where(trades_df.signal != 0, 1, 0)
    
    #fees = (_fees * _position_size) * attempt_trade
    
    trades_df['pl'] = np.where(win == 1, ((_position_size * _tp) - _fees) - _position_size, 0)
    trades_df['pl'] = np.where(lose == 1, ((_position_size * _stop) - _fees) - _position_size, trades_df.pl)
    
    trades_df['cum_pl'] = trades_df.pl.cumsum()
    
    
    wins = sum(win)
    losses = sum(lose)
    win_rate = wins/ (wins+losses)
    
    win_size = ((_tp - 1) - _fees) * 100
    loss_size = ((1 - _stop) + _fees) * 100
    RR = win_size/loss_size
    
    print('Wins: ' + str(wins))
    print('Losses: ' + str(losses))
    print('Win rate: ' + str(win_rate))
    print('Win size: ' + str(win_size))
    print('Loss size: ' + str(loss_size))
    print('RR: ' + str(RR))



    
    trades_df.cum_pl.plot()
    
    return trades_df

analysis = analyse_results(trades, 0.0015, 1000, 0.99, 1.013)
analysis.tail()




Wins: 1067
Losses: 1372
Win rate: 0.43747437474374745
Win size: 1.1499999999999901
Loss size: 1.1500000000000008
RR: 0.9999999999999908


,ts,o,h,l,c,signal,result,pl,cum_pl
10564,2019-06-14 21:40:00,262.15,265.9,262.15,264.50,0,1,12.9985,147.3415
10565,2019-06-15 01:25:00,264.75,265.7,264.80,265.45,0,0,0.0000,147.3415
10566,2019-06-15 01:25:00,264.75,265.7,264.80,265.45,0,0,0.0000,147.3415
10567,2019-06-15 01:25:00,264.75,265.7,264.80,265.45,0,0,0.0000,147.3415
10568,2019-06-15 01:25:00,264.75,265.7,264.80,265.45,0,0,0.0000,147.3415
